# Esercizio 3 - Hanks

## Idea
- Il verbo scelto è **Love**
- Si estrae dal corpus **Brown** le istanze che contengono il verbo **Love**
- Si effettua parsing delle frasi estratte per cercare soggetti e oggetti relativi al verbo **Love**
- Si cercano i super-sensi dei soggetti e degli oggetti
- Si aggregano i risultati ottenuti

In [128]:
import nltk
from nltk.corpus import brown
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import spacy

### Estrai le frasi che contengono il verbo Love dal corpus Brown

In [134]:
nlp = spacy.load('en_core_web_sm')
verb = ['love', 'loves', 'loved']
sentences = [sent for sent in brown.sents() if verb[0] in sent or verb[1] in sent or verb[2] in sent]
print(len(sentences))

271


### Parsing delle frasi estratte per trovare il soggetto e l'oggetto del verbo Love

Per ogni frase estratta dal corpus cerco il soggetto e l'oggetto del verbo **Love** e li salvo in una lista di tuple.  
Per farlo uso la libreria **spaCy** che mi permette di cercare il soggetto e l'oggetto di un verbo in una frase tramite `token.dep_ == 'nsubj'` e `token.dep_ == 'dobj'`.  
Tramite `token.head.text in verb` posso inoltre cercare se il soggetto e l'oggetto trovati sono effettivamente legati al verbo **Love**.  
Restituisco una lista di triplette (soggetto, oggetto, frase) perché mi servirà la frase per disambiguare il soggetto e l'oggetto.

In [135]:
# Parse the sentences using Spacy to extract the subject and object of the verb love
def get_subject_object(sentence):
    triples = []
    for sent in sentences:
        sub = ''
        obj = ''
        doc = nlp(' '.join(sent))
        for token in doc:
            if token.dep_ == 'nsubj' and token.head.text in verb:
                sub = token.text
            if token.dep_ == 'dobj' and token.head.text in verb:
                obj = token.text
        if sub != '' and obj != '':
            triples.append((sub, obj, sent))
    return triples

### Disambiguazione delle coppie soggetto-oggetto

Per ogni coppia soggetto-oggetto estratta dal corpus, cerco il super-senso di WordNet che meglio descrive il soggetto e l'oggetto. Per farlo utilizzo l'algoritmo di Lesk tramite la libreria **nltk**.  
Se non trovo nessun super-senso per il soggetto o l'oggetto, inserisco di default "noun.person".

In [136]:
# Get the super senses of the subject and object
def get_super_sense(triples):
    super_senses = []
    for triple in triples:
        sub = triple[0]
        obj = triple[1]
        sent = triple[2]
        sub_sense = lesk(sent, sub)
        obj_sense = lesk(sent, obj)
        super_senses.append((sub_sense.lexname() if sub_sense else 'noun.person', obj_sense.lexname() if obj_sense else 'noun.person'))
    return super_senses

### Aggregazione dei risultati

In [137]:
# Aggregate the super senses
def aggregate_super_sense(super_senses):
    super_sense_dict = {}
    for super_sense in super_senses:
        if super_sense in super_sense_dict:
            super_sense_dict[super_sense] += 1
        else:
            super_sense_dict[super_sense] = 1
    super_sense_dict = {k: v for k, v in sorted(super_sense_dict.items(), key=lambda item: item[1], reverse=True)}
    return super_sense_dict

result = aggregate_super_sense(get_super_sense(get_subject_object(sentences)))
total = sum(result.values())
print(f'Different pairs: {len(result)}')
for key, value in result.items():
    print(f'{key}: {value/total*100:.2f}%')

Different pairs: 35
('noun.person', 'noun.person'): 22.22%
('noun.substance', 'noun.person'): 12.50%
('adj.all', 'noun.person'): 5.56%
('noun.person', 'noun.cognition'): 4.17%
('noun.group', 'noun.person'): 4.17%
('noun.person', 'noun.group'): 4.17%
('noun.person', 'noun.attribute'): 2.78%
('noun.substance', 'noun.group'): 2.78%
('adj.all', 'verb.creation'): 2.78%
('noun.substance', 'noun.location'): 2.78%
('noun.person', 'noun.location'): 2.78%
('noun.group', 'verb.body'): 1.39%
('noun.person', 'verb.motion'): 1.39%
('noun.person', 'noun.Tops'): 1.39%
('noun.group', 'verb.cognition'): 1.39%
('noun.group', 'noun.location'): 1.39%
('noun.substance', 'verb.emotion'): 1.39%
('noun.group', 'noun.cognition'): 1.39%
('adj.all', 'verb.communication'): 1.39%
('noun.substance', 'noun.cognition'): 1.39%
('noun.person', 'noun.artifact'): 1.39%
('noun.group', 'noun.feeling'): 1.39%
('verb.stative', 'noun.location'): 1.39%
('noun.state', 'noun.cognition'): 1.39%
('noun.substance', 'noun.artifact'):